In [ ]:
import pandas
import numpy
from matplotlib import pyplot
filename = "dataframe.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = data.index.astype(float)
xmax, xmin = 2000, 1000
data = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
data.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()

In [ ]:
# スペクトル表示
from ipywidgets import Play, IntSlider, jslink, HBox, interactive_output
x = data.columns
xmin, xmax = x.min(), x.max()
ymin, ymax = data.min().min(), data.max().max()
def f(i):
  y = data.iloc[i]
  pyplot.xlim(xmax, xmin)
  pyplot.ylim(ymin, ymax)
  pyplot.plot(x, y)
  pyplot.show()
play = Play(min=0, max=len(data) - 1, step=1, interval=100)
slider = IntSlider(min=0, max=len(data) - 1, step=1)
jslink((play, "value"), (slider, "value"))
display(HBox([play, slider]), interactive_output(f, {"i": slider}))

In [ ]:
# 信号強度変化
from ipywidgets import IntSlider, interactive_output
x = data.columns
s = data.index
y = data.values.T
xmin, xmax = x.min(), x.max()
smin, smax = s.min(), s.max()
ymin, ymax = y.min(), y.max()
def f(i):
	j = len(data.T) - 1 - i
	print("i =", j, " pos =", x[j])
	pyplot.xlim(xmax, xmin)
	pyplot.ylim(ymin, ymax)
	pyplot.plot(x, y)
	pyplot.plot([x[j], x[j]], [ymin, ymax])
	pyplot.show()
	pyplot.xlim(smin, smax)
	pyplot.ylim(ymin, ymax)
	pyplot.plot(s, data.iloc[:,j].values)
	pyplot.show()
slider = IntSlider(min=0, max=len(data.T)-1, step=1)
display(slider, interactive_output(f, {"i": slider}))

In [ ]:
# 信号強度変化の保存
j = len(data.T) - 1 - slider.get_interact_value()
print("i =", j, " pos =", x[j])
buff = pandas.Series(data.iloc[:,j].values, index=data.index)
buff.plot()
pyplot.show()
#buff.to_csv(str(x[j])+".csv", header=False)

In [ ]:
# 面積強度の保存
left, right = 36, 27
print(data.columns[left+1], "-", data.columns[right])
buff = data.iloc[:,right:left+1].T.sum()
buff.plot()
pyplot.show()
#buff.to_csv(str(x[left])+"_"+str(x[right])+".csv", header=False)

In [ ]:
# 平滑化
from ipywidgets import IntSlider, interactive_output
from scipy.signal import savgol_filter
x = data.columns
y = data.values[-1]
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
def f(i):
  buff = savgol_filter(y, i, 2, 0)
  pyplot.xlim(xmax, xmin)
  pyplot.ylim(ymin, ymax)
  pyplot.scatter(x, y, s=5, c="blue")
  pyplot.plot(x, buff, c="red")
  pyplot.show()
  pyplot.xlim(xmax, xmin)
  pyplot.ylim((ymin - ymax) / 10, (ymax - ymin) / 10)
  pyplot.scatter(x, y - buff, s=5, c="green")
  pyplot.show()
secder = False
slider = IntSlider(min=3, max=201, step=2)
display(slider, interactive_output(f, {"i": slider}))

In [ ]:
# 二次微分
from ipywidgets import IntSlider, interactive_output
from scipy.signal import savgol_filter
x = data.columns
y = data.values[-1]
buff = savgol_filter(y, 3, 2, 2)
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
bmin, bmax = buff.min(), buff.max()
def f(i):
  buff = savgol_filter(y, i, 2, 0)
  pyplot.xlim(xmax, xmin)
  pyplot.ylim(ymin, ymax)
  pyplot.scatter(x, y, s=5, c="blue")
  pyplot.plot(x, buff, c="red")
  pyplot.show()
  buff = savgol_filter(buff, i, 2, 1)
  buff = savgol_filter(buff, i, 2, 0)
  buff = savgol_filter(buff, i, 2, 1)
  pyplot.xlim(xmax, xmin)
  pyplot.ylim(bmin, bmax)
  pyplot.plot(x, buff, c="red")
  pyplot.show()
secder = True
slider = IntSlider(min=3, max=201, step=2)
display(slider, interactive_output(f, {"i": slider}))

In [ ]:
# 全てのスペクトルを平滑化 / 二次微分
window = slider.get_interact_value()
print("window size =", x[window - 1] - x[0])
buff = data
buff = savgol_filter(buff, window, 2, 0)
if secder:
  buff = savgol_filter(buff, window, 2, 1)
  buff = savgol_filter(buff, window, 2, 0)
  buff = savgol_filter(buff, window, 2, 1)
buff = pandas.DataFrame(buff, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
pyplot.show()
#data = buff